In [1]:
import sys

In [2]:
txt_data = [line.rstrip('\n').rstrip(')').lstrip('(') for line in open('GraphData.txt')]
graph_data = []

# Put graph data into a array of tuples.
for line in txt_data:
    split = line.split(',')
    if len(split) == 2:
        x = int(split[0])
        y = int(split[1])
        graph_data.append((x, y, abs(x - y)))
        
print(graph_data)

[(277, 325, 48), (6499, 6525, 26), (1586, 1688, 102), (1435, 1500, 65), (2817, 2954, 137), (1467, 1550, 83), (4952, 4986, 34), (4393, 4449, 56), (840, 895, 55), (5088, 5154, 66), (3637, 3744, 107), (3618, 3622, 4), (2287, 2422, 135), (5143, 5184, 41), (6468, 6472, 4), (2167, 2183, 16), (3557, 3706, 149), (1441, 1507, 66), (84, 210, 126), (5625, 5870, 245), (6389, 6421, 32), (2279, 2421, 142), (1513, 1528, 15), (5016, 5078, 62), (1634, 1698, 64), (4215, 4474, 259), (4415, 4455, 40), (6327, 6568, 241), (5136, 5175, 39), (3054, 3131, 77), (3553, 3641, 88), (5123, 5138, 15), (928, 1027, 99), (2869, 3036, 167), (41, 144, 103), (4969, 5016, 47), (250, 322, 72), (6461, 6494, 33), (4964, 5096, 132), (3098, 3104, 6), (4242, 4460, 218), (3652, 3720, 68), (31, 264, 233), (740, 902, 162), (1437, 1658, 221), (3582, 3744, 162), (826, 898, 72), (2382, 2433, 51), (1594, 1680, 86), (5670, 5744, 74), (5682, 5749, 67), (5634, 5853, 219), (1509, 1691, 182), (4990, 5019, 29), (915, 1024, 109), (2811, 3054,

In [3]:
class Forest():
    """
    Class to take in an unconnected list of tuples containing (vert1, vert2, weight) data.
    Split it into individual connected components.
    Convert the node labels into integers (0 ... n).
    Create an adjacency list for graph.
    Do the same for each found subgraph in the data.
    """
    def __init__(self, graph_data):
        """
        Initializes a new instance of the Forest class
        ----------
        Parameters
            graph_data : [(int, int, int)]
                The graph data to build the dict off of.
        """          
        self.graph_data = graph_data

        dicts = self.build_graph_dict(self.graph_data)        
        self.graph_dict = dicts[0]
        self.graph_dict_rev = dicts[1]                
        
        self.V = len(self.graph_dict)
        
        self.graph = self.build_adj_list(self.graph_data, self.graph_dict, self.V)
        self.cc = self.get_connected_components()
        
        self.forest_dicts = []
        self.forest = []
        self.forest_data = []
        self.build_connected_forest()

        print("Forest Info:")
        print("\tNumber of vertices:", str(self.V))
        print("\tNumber of edges:", len(self.graph_data))
        print("\tNumber of connected components in forest:", len(self.cc))
        
        for i in range(len(self.forest)):
            print("\t\tSubgraph {0}: ".format(i))
            print("\t\t\t# vertices:", len(self.forest_dicts[i][0]))            
            print("\t\t\t# edges:", len(self.forest_data[i]))            
        
    def build_graph_dict(self, graph_data):
        """
        Builds a dictionary containing the labels in the graph.
        ----------
        Parameters
            graph_data : [(int, int, int)]
                The graph data to build the dict off of.
        """        
        graph_dict = {}
        graph_dict_rev = {}
        i = 0

        for x in graph_data:
            if x[0] not in graph_dict:
                graph_dict[x[0]] = i
                graph_dict_rev[i] = x[0]
                i += 1
            if x[1] not in graph_dict:
                graph_dict[x[1]] = i
                graph_dict_rev[i] = x[1]
                i += 1

        return (graph_dict, graph_dict_rev)
        
    def build_adj_list(self, graph_data, graph_dict, length):
        """
        Builds an adjacency list of the given graph data.
        ----------
        Parameters
            graph_data : [(int, int, int)]
                The graph data to build the adj list off of.
            graph_dict : {}
                The graph data's dict.
            length : int
                The number of nodes in the graph.
        """          
        graph = [[(0, 0) for i in range(length)] for j in range(length)]

        for d in graph_data:
            x = graph_dict[d[0]]
            y = graph_dict[d[1]]
            graph[x][y] = (y, d[2])
            graph[y][x] = (x, d[2])
        
        return graph
            
    def build_connected_forest(self):
        """
        Builds an array of graphs from the separate components
        in the class's total graph data.
        """                  
        for c in self.cc:
            graph_data = []
            for x in c:
                for edge in self.find_edges(x):
                    if not self.edge_in_edges(edge, graph_data):
                        graph_data.append(edge)   

            d = self.build_graph_dict(graph_data)
            adj = self.build_adj_list(graph_data, d[0], len(d[0]))
            
            self.forest_dicts.append(d)
            self.forest_data.append(graph_data)
            self.forest.append(adj)
                
    def find_edges(self, v):
        """
        Gets the edges matching the given vertex from the graph data.
        ----------
        Parameters
            v : int
                The requested vertex.
        """                  
        edges = []
        for edge in self.graph_data:
            if edge[0] == self.graph_dict_rev[v] or edge[1] == self.graph_dict_rev[v]:
                edges.append(edge)

        return edges
    
    def edge_in_edges(self, edge, edges):
        """
        Checks if an edge already exists in the edges list.
        ----------
        Parameters
            edge : (int, int)
                The requested edge.
            edge : [(int, int)]
                The list of edges.
        """          
        for e in edges:
            if edge[0] == e[0] and edge[1] == e[1]:
                return True

        return False
            
    def print_cc_list(self):
        """
        Prints out the separate componets from the forest.
        """          
        i = 0
        for cc in self.forest_data:
            print("\nComponent {0}:".format(str(i)))
            i += 1
            for c in cc:
                s = "["
                for x in c:
                    s += str(x) + ','
                s = s[:-1]
                s += "]"
                print(s)        
                
    def get_connected_components(self): 
        """
        Uses DFS to find connected components in the total graph.
        """           
        cc = [] 
        repeat = [] 
        
        for i in range(self.V): 
            repeat.append(False) 

        for v in range(self.V): 
            if not repeat[v]: 
                t = [] 
                cc.append(self.dfs(t, v, repeat)) 
        
        return cc 
    
    def dfs(self, t, v, repeat):
        """
        Uses the DFS algorithm to recursively scan through the graph looking for connected edges.
        ----------
        Parameters
            t : []
                The temp list used for checking existing values.
            v : int
                The new vertex to add.
            repeat : []
                The list of vertices that already exist in the component.
        """         
        repeat[v] = True
        t.append(v) 

        for i in self.graph[v]: 
            if not repeat[i[0]]: 
                t = self.dfs(t, i[0], repeat) 

        return t

In [13]:
import sys 
  
class Graph():   
    """
    Class to contain the algorithms for MST-Prim and graph data.
    Graph data should be stored a an adjacency list with the matching nodes containing the weight of the edge.
    """
    def __init__(self, vertices, graph_dict):
        """
        Initializes a new instance of the Graph class
        ----------
        Parameters
            graph_dict : {}
                The graph dictionary containing the node labels.
        """        
        self.V = vertices 
        self.graph = [[0 for column in range(vertices)] for row in range(vertices)] 
        self.graph_dict = graph_dict

    def print_all_pairs_shortest_path(self, s, i): 
        """
        Prints out the path length for each node in the graph.
        ----------
        Parameters
            s : int
                The starting node.
            i : []
                The starting node paths.
        """     
        print("(v1 -> v2) - Weighted Distance")
        max_weight = 0
        max_str = ""
        for node in range(self.V): 
            out = "({0} -> {1}) - {2}".format(self.graph_dict[s], self.graph_dict[node], i[node])
            print(out)
            if i[node] > max_weight:
                max_weight = i[node]
                max_str = out
        print("Max Path:", max_str)

    def extract_min(self, w, S): 
        """
        Gets the minimum weighted node from the start to each of the graph vertices.
        ----------
        Parameters
            w : int
                The starting node weights.
            S : []
                The set of node paths.
        """     
        min = sys.maxsize 

        for v in range(self.V): 
            if w[v] < min and S[v] == False: 
                min = w[v] 
                min_index = v 

        return min_index 

    def dijkstra(self, s): 
        """
        Preforms the Disjkstra Shortest Paths algorithm from the book p.658 (... and Wikipedia)
        ----------
        Parameters
            w : int
                The shortest path weight.
            s : []
                The source node.
        """           
        # Essentially INITIALIZE-SINGLE-SOURCE
        w = [sys.maxsize] * self.V 
        S = [False] * self.V 
        w[s] = 0

        for i in range(self.V): 
            u = self.extract_min(w, S) 
            S[u] = True

            # For each Vertex in Adj Graph
            for v in range(self.V): 
                if self.graph[u][v][1] > 0 and not S[v]:
                    if w[v] > w[u] + self.graph[u][v][1]: 
                        w[v] = w[u] + self.graph[u][v][1] 

        self.print_all_pairs_shortest_path(s, w) 

In [14]:
forest = Forest(graph_data)
print("")
print("Dijkstra Shortest-Paths on Subgraph:", "1")
verticies = len(forest.forest_dicts[1][0])

for v in range(verticies):
    print("Path starting at vertex:", forest.forest_dicts[1][1][v])
    data = forest.forest[1]
    graph_dict = forest.forest_dicts[1][1]
    g = Graph(verticies, graph_dict)
    g.graph = data
    g.dijkstra(v)
    print("")

Forest Info:
	Number of vertices: 1426
	Number of edges: 2342
	Number of connected components in forest: 10
		Subgraph 0: 
			# vertices: 159
			# edges: 267
		Subgraph 1: 
			# vertices: 125
			# edges: 199
		Subgraph 2: 
			# vertices: 161
			# edges: 271
		Subgraph 3: 
			# vertices: 158
			# edges: 265
		Subgraph 4: 
			# vertices: 125
			# edges: 199
		Subgraph 5: 
			# vertices: 125
			# edges: 199
		Subgraph 6: 
			# vertices: 162
			# edges: 273
		Subgraph 7: 
			# vertices: 125
			# edges: 199
		Subgraph 8: 
			# vertices: 161
			# edges: 271
		Subgraph 9: 
			# vertices: 125
			# edges: 199

Dijkstra Shortest-Paths on Subgraph: 1
Path starting at vertex: 6499
(v1 -> v2) - Weighted Distance
(6499 -> 6499) - 0
(6499 -> 6525) - 26
(6499 -> 6590) - 91
(6499 -> 6576) - 77
(6499 -> 6414) - 239
(6499 -> 6476) - 177
(6499 -> 6540) - 113
(6499 -> 6567) - 86
(6499 -> 6502) - 151
(6499 -> 6581) - 82
(6499 -> 6566) - 87
(6499 -> 6354) - 299
(6499 -> 6417) - 236
(6499 -> 6368) - 285
(6499

(6581 -> 6520) - 191
(6581 -> 6337) - 244
(6581 -> 6473) - 280
(6581 -> 6372) - 209
(6581 -> 6434) - 147
(6581 -> 6463) - 288
(6581 -> 6326) - 333
(6581 -> 6421) - 238
(6581 -> 6389) - 270
(6581 -> 6388) - 271
(6581 -> 6352) - 307
(6581 -> 6408) - 289
(6581 -> 6486) - 333
(6581 -> 6482) - 425
(6581 -> 6359) - 460
(6581 -> 6427) - 392
(6581 -> 6399) - 182
(6581 -> 6539) - 252
(6581 -> 6314) - 267
(6581 -> 6415) - 198
(6581 -> 6347) - 300
(6581 -> 6472) - 291
(6581 -> 6468) - 287
(6581 -> 6381) - 200
(6581 -> 6478) - 103
(6581 -> 6308) - 273
(6581 -> 6336) - 245
(6581 -> 6363) - 218
(6581 -> 6584) - 319
(6581 -> 6435) - 170
(6581 -> 6545) - 280
(6581 -> 6423) - 158
(6581 -> 6491) - 226
(6581 -> 6548) - 163
(6581 -> 6315) - 396
(6581 -> 6577) - 206
(6581 -> 6365) - 216
(6581 -> 6313) - 470
(6581 -> 6551) - 376
(6581 -> 6443) - 308
(6581 -> 6470) - 399
(6581 -> 6331) - 292
(6581 -> 6490) - 91
(6581 -> 6411) - 248
(6581 -> 6572) - 307
(6581 -> 6380) - 335
(6581 -> 6526) - 261
(6581 -> 6518)

(6374 -> 6327) - 329
(6374 -> 6343) - 437
(6374 -> 6344) - 346
(6374 -> 6500) - 210
(6374 -> 6496) - 160
(6374 -> 6440) - 296
(6374 -> 6555) - 181
(6374 -> 6383) - 269
(6374 -> 6580) - 206
(6374 -> 6457) - 83
(6374 -> 6452) - 284
(6374 -> 6377) - 237
(6374 -> 6504) - 130
(6374 -> 6461) - 153
(6374 -> 6494) - 120
(6374 -> 6409) - 205
(6374 -> 6317) - 57
(6374 -> 6465) - 91
(6374 -> 6307) - 67
(6374 -> 6333) - 41
(6374 -> 6387) - 13
(6374 -> 6379) - 139
(6374 -> 6367) - 173
(6374 -> 6312) - 228
(6374 -> 6527) - 153
(6374 -> 6515) - 141
(6374 -> 6479) - 155
(6374 -> 6453) - 181
(6374 -> 6462) - 218
(6374 -> 6565) - 191
(6374 -> 6330) - 304
(6374 -> 6397) - 123
(6374 -> 6342) - 224
(6374 -> 6520) - 224
(6374 -> 6337) - 63
(6374 -> 6473) - 99
(6374 -> 6372) - 28
(6374 -> 6434) - 60
(6374 -> 6463) - 179
(6374 -> 6326) - 246
(6374 -> 6421) - 279
(6374 -> 6389) - 311
(6374 -> 6388) - 246
(6374 -> 6352) - 340
(6374 -> 6408) - 330
(6374 -> 6486) - 270
(6374 -> 6482) - 362
(6374 -> 6359) - 397
(6

(6405 -> 6352) - 445
(6405 -> 6408) - 445
(6405 -> 6486) - 385
(6405 -> 6482) - 477
(6405 -> 6359) - 512
(6405 -> 6427) - 438
(6405 -> 6399) - 300
(6405 -> 6539) - 332
(6405 -> 6314) - 385
(6405 -> 6415) - 316
(6405 -> 6347) - 352
(6405 -> 6472) - 477
(6405 -> 6468) - 481
(6405 -> 6381) - 434
(6405 -> 6478) - 463
(6405 -> 6308) - 391
(6405 -> 6336) - 427
(6405 -> 6363) - 578
(6405 -> 6584) - 179
(6405 -> 6435) - 328
(6405 -> 6545) - 438
(6405 -> 6423) - 340
(6405 -> 6491) - 408
(6405 -> 6548) - 291
(6405 -> 6315) - 384
(6405 -> 6577) - 494
(6405 -> 6365) - 504
(6405 -> 6313) - 758
(6405 -> 6551) - 590
(6405 -> 6443) - 422
(6405 -> 6470) - 485
(6405 -> 6331) - 406
(6405 -> 6490) - 325
(6405 -> 6411) - 442
(6405 -> 6572) - 489
(6405 -> 6380) - 417
(6405 -> 6526) - 443
(6405 -> 6518) - 279
(6405 -> 6560) - 477
(6405 -> 6436) - 507
(6405 -> 6556) - 269
Max Path: (6405 -> 6313) - 758

Path starting at vertex: 6538
(v1 -> v2) - Weighted Distance
(6538 -> 6499) - 179
(6538 -> 6525) - 153
(653

(6376 -> 6452) - 398
(6376 -> 6377) - 319
(6376 -> 6504) - 242
(6376 -> 6461) - 275
(6376 -> 6494) - 242
(6376 -> 6409) - 327
(6376 -> 6317) - 179
(6376 -> 6465) - 213
(6376 -> 6307) - 189
(6376 -> 6333) - 81
(6376 -> 6387) - 135
(6376 -> 6379) - 261
(6376 -> 6367) - 287
(6376 -> 6312) - 342
(6376 -> 6527) - 265
(6376 -> 6515) - 139
(6376 -> 6479) - 217
(6376 -> 6453) - 293
(6376 -> 6462) - 330
(6376 -> 6565) - 303
(6376 -> 6330) - 366
(6376 -> 6397) - 245
(6376 -> 6342) - 102
(6376 -> 6520) - 176
(6376 -> 6337) - 185
(6376 -> 6473) - 221
(6376 -> 6372) - 150
(6376 -> 6434) - 182
(6376 -> 6463) - 231
(6376 -> 6326) - 318
(6376 -> 6421) - 361
(6376 -> 6389) - 393
(6376 -> 6388) - 358
(6376 -> 6352) - 430
(6376 -> 6408) - 412
(6376 -> 6486) - 286
(6376 -> 6482) - 354
(6376 -> 6359) - 413
(6376 -> 6427) - 227
(6376 -> 6399) - 179
(6376 -> 6539) - 287
(6376 -> 6314) - 62
(6376 -> 6415) - 163
(6376 -> 6347) - 29
(6376 -> 6472) - 154
(6376 -> 6468) - 158
(6376 -> 6381) - 245
(6376 -> 6478) -

...

(6326 -> 6444) - 414
(6326 -> 6552) - 306
(6326 -> 6405) - 453
(6326 -> 6538) - 296
(6326 -> 6328) - 530
(6326 -> 6536) - 290
(6326 -> 6493) - 365
(6326 -> 6394) - 164
(6326 -> 6402) - 156
(6326 -> 6585) - 339
(6326 -> 6534) - 292
(6326 -> 6376) - 318
(6326 -> 6587) - 261
(6326 -> 6442) - 116
(6326 -> 6516) - 332
(6326 -> 6428) - 102
(6326 -> 6530) - 268
(6326 -> 6568) - 280
(6326 -> 6327) - 305
(6326 -> 6343) - 505
(6326 -> 6344) - 288
(6326 -> 6500) - 448
(6326 -> 6496) - 248
(6326 -> 6440) - 192
(6326 -> 6555) - 307
(6326 -> 6383) - 135
(6326 -> 6580) - 320
(6326 -> 6457) - 287
(6326 -> 6452) - 410
(6326 -> 6377) - 449
(6326 -> 6504) - 244
(6326 -> 6461) - 391
(6326 -> 6494) - 366
(6326 -> 6409) - 339
(6326 -> 6317) - 303
(6326 -> 6465) - 337
(6326 -> 6307) - 313
(6326 -> 6333) - 287
(6326 -> 6387) - 233
(6326 -> 6379) - 365
(6326 -> 6367) - 377
(6326 -> 6312) - 432
(6326 -> 6527) - 289
(6326 -> 6515) - 229
(6326 -> 6479) - 269
(6326 -> 6453) - 295
(6326 -> 6462) - 354
(6326 -> 6565

(6427 -> 6482) - 151
(6427 -> 6359) - 186
(6427 -> 6427) - 0
(6427 -> 6399) - 304
(6427 -> 6539) - 164
(6427 -> 6314) - 289
(6427 -> 6415) - 320
(6427 -> 6347) - 256
(6427 -> 6472) - 313
(6427 -> 6468) - 309
(6427 -> 6381) - 222
(6427 -> 6478) - 447
(6427 -> 6308) - 295
(6427 -> 6336) - 611
(6427 -> 6363) - 600
(6427 -> 6584) - 439
(6427 -> 6435) - 350
(6427 -> 6545) - 372
(6427 -> 6423) - 362
(6427 -> 6491) - 420
(6427 -> 6548) - 399
(6427 -> 6315) - 492
(6427 -> 6577) - 412
(6427 -> 6365) - 422
(6427 -> 6313) - 434
(6427 -> 6551) - 438
(6427 -> 6443) - 160
(6427 -> 6470) - 457
(6427 -> 6331) - 272
(6427 -> 6490) - 325
(6427 -> 6411) - 270
(6427 -> 6572) - 501
(6427 -> 6380) - 309
(6427 -> 6526) - 455
(6427 -> 6518) - 391
(6427 -> 6560) - 489
(6427 -> 6436) - 373
(6427 -> 6556) - 413
Max Path: (6427 -> 6336) - 611

Path starting at vertex: 6399
(v1 -> v2) - Weighted Distance
(6399 -> 6499) - 254
(6399 -> 6525) - 228
(6399 -> 6590) - 223
(6399 -> 6576) - 177
(6399 -> 6414) - 339
(6399 

(6308 -> 6534) - 226
(6308 -> 6376) - 68
(6308 -> 6587) - 375
(6308 -> 6442) - 134
(6308 -> 6516) - 446
(6308 -> 6428) - 334
(6308 -> 6530) - 374
(6308 -> 6568) - 394
(6308 -> 6327) - 395
(6308 -> 6343) - 619
(6308 -> 6344) - 412
(6308 -> 6500) - 400
(6308 -> 6496) - 226
(6308 -> 6440) - 442
(6308 -> 6555) - 363
(6308 -> 6383) - 385
(6308 -> 6580) - 382
(6308 -> 6457) - 265
(6308 -> 6452) - 466
(6308 -> 6377) - 383
(6308 -> 6504) - 306
(6308 -> 6461) - 343
(6308 -> 6494) - 310
(6308 -> 6409) - 395
(6308 -> 6317) - 247
(6308 -> 6465) - 281
(6308 -> 6307) - 257
(6308 -> 6333) - 149
(6308 -> 6387) - 203
(6308 -> 6379) - 329
(6308 -> 6367) - 355
(6308 -> 6312) - 410
(6308 -> 6527) - 329
(6308 -> 6515) - 207
(6308 -> 6479) - 281
(6308 -> 6453) - 357
(6308 -> 6462) - 394
(6308 -> 6565) - 289
(6308 -> 6330) - 430
(6308 -> 6397) - 313
(6308 -> 6342) - 34
(6308 -> 6520) - 240
(6308 -> 6337) - 253
(6308 -> 6473) - 289
(6308 -> 6372) - 218
(6308 -> 6434) - 250
(6308 -> 6463) - 299
(6308 -> 6326) 

(6577 -> 6330) - 447
(6577 -> 6397) - 300
(6577 -> 6342) - 427
(6577 -> 6520) - 387
(6577 -> 6337) - 240
(6577 -> 6473) - 276
(6577 -> 6372) - 205
(6577 -> 6434) - 143
(6577 -> 6463) - 356
(6577 -> 6326) - 423
(6577 -> 6421) - 430
(6577 -> 6389) - 398
(6577 -> 6388) - 449
(6577 -> 6352) - 361
(6577 -> 6408) - 417
(6577 -> 6486) - 353
(6577 -> 6482) - 445
(6577 -> 6359) - 462
(6577 -> 6427) - 412
(6577 -> 6399) - 378
(6577 -> 6539) - 248
(6577 -> 6314) - 263
(6577 -> 6415) - 364
(6577 -> 6347) - 296
(6577 -> 6472) - 415
(6577 -> 6468) - 411
(6577 -> 6381) - 396
(6577 -> 6478) - 299
(6577 -> 6308) - 393
(6577 -> 6336) - 441
(6577 -> 6363) - 414
(6577 -> 6584) - 315
(6577 -> 6435) - 166
(6577 -> 6545) - 276
(6577 -> 6423) - 154
(6577 -> 6491) - 222
(6577 -> 6548) - 359
(6577 -> 6315) - 592
(6577 -> 6577) - 0
(6577 -> 6365) - 212
(6577 -> 6313) - 264
(6577 -> 6551) - 502
(6577 -> 6443) - 376
(6577 -> 6470) - 555
(6577 -> 6331) - 450
(6577 -> 6490) - 287
(6577 -> 6411) - 244
(6577 -> 6572) 

(6572 -> 6382) - 328
(6572 -> 6420) - 554
(6572 -> 6375) - 545
(6572 -> 6554) - 592
(6572 -> 6492) - 482
(6572 -> 6300) - 480
(6572 -> 6305) - 475
(6572 -> 6444) - 434
(6572 -> 6552) - 418
(6572 -> 6405) - 489
(6572 -> 6538) - 404
(6572 -> 6328) - 642
(6572 -> 6536) - 402
(6572 -> 6493) - 477
(6572 -> 6394) - 544
(6572 -> 6402) - 462
(6572 -> 6585) - 397
(6572 -> 6534) - 400
(6572 -> 6376) - 404
(6572 -> 6587) - 455
(6572 -> 6442) - 500
(6572 -> 6516) - 526
(6572 -> 6428) - 422
(6572 -> 6530) - 398
(6572 -> 6568) - 474
(6572 -> 6327) - 399
(6572 -> 6343) - 537
(6572 -> 6344) - 416
(6572 -> 6500) - 226
(6572 -> 6496) - 442
(6572 -> 6440) - 386
(6572 -> 6555) - 463
(6572 -> 6383) - 329
(6572 -> 6580) - 488
(6572 -> 6457) - 365
(6572 -> 6452) - 566
(6572 -> 6377) - 519
(6572 -> 6504) - 412
(6572 -> 6461) - 435
(6572 -> 6494) - 402
(6572 -> 6409) - 487
(6572 -> 6317) - 339
(6572 -> 6465) - 191
(6572 -> 6307) - 349
(6572 -> 6333) - 323
(6572 -> 6387) - 291
(6572 -> 6379) - 421
(6572 -> 6367